In [1]:
# import library
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
import os
import plotly.express as px
import geopandas as gpd
import folium
import branca
from tqdm.notebook import tqdm
from folium import Marker, FeatureGroup
from folium.plugins import MarkerCluster, FeatureGroupSubGroup, Search

In [2]:
# get the current working directory
cwd = os.getcwd()

# print the current working directory
print(f"Current working directory: {cwd}")

path = '/Users/johnn/Documents/Research/codebook/covid/'
os.chdir(path)

# get the current working directory
cwd = os.getcwd()
# print the current working directory
print(f"Current working directory: {cwd}")


Current working directory: c:\Users\johnn\Documents\Research\codebook\covid
Current working directory: c:\Users\johnn\Documents\Research\codebook\covid


In [3]:
# define global parameters for web scrapping.
headers = {'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36'}

In [4]:
# draw map
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='100px')

In [5]:
# request address
def transformer(geo):
    parameters = {'address': geo, 'key': '38f20eb40106630346281a14392f3629', 'city':'shanghai', 'batch':'true'}
    base = 'https://restapi.amap.com/v3/geocode/geo'
    try:
        response = requests.get(base, parameters, timeout=30)
        answer = response.json()
        return answer
    except:
        return None

In [6]:
# load link table
result_sorted = pd.read_csv('link_all.csv')
result_sorted.drop(columns='Unnamed: 0', inplace=True)

In [7]:
date = pd.to_datetime('2022-04-26', format='%Y-%m-%d')
link = 'https://mp.weixin.qq.com/s/SIuDbITNdgWwYyM3eiyrgg'
#date = pd.to_datetime('2022-04-09', format='%Y-%m-%d')

In [8]:
result_sorted = pd.concat([result_sorted, pd.DataFrame({"date": [date], "link": [link]})], ignore_index=True)

In [9]:
result_sorted

,date,link
0,2022-03-18,http://wsjkw.sh.gov.cn/xwfb/20220319/dc5938b3d...
1,2022-03-19,http://wsjkw.sh.gov.cn/xwfb/20220320/f9f1683cf...
2,2022-03-20,http://wsjkw.sh.gov.cn/xwfb/20220321/2cda1b24b...
3,2022-03-22,http://wsjkw.sh.gov.cn/xwfb/20220323/dd54a58cd...
4,2022-03-23,http://wsjkw.sh.gov.cn/xwfb/20220324/a8643ae53...
5,2022-03-24,http://wsjkw.sh.gov.cn/xwfb/20220325/a76fbb6a1...
6,2022-03-25,http://wsjkw.sh.gov.cn/xwfb/20220326/ff80e7e71...
7,2022-03-26,http://wsjkw.sh.gov.cn/xwfb/20220327/09cfe78de...
8,2022-03-27,http://wsjkw.sh.gov.cn/xwfb/20220328/12b074e39...
9,2022-03-28,http://wsjkw.sh.gov.cn/xwfb/20220329/b7430a2b3...


In [10]:
result_sorted['date'] = pd.to_datetime(result_sorted['date'])

In [11]:
result_sorted.to_csv('link_all.csv')

In [12]:
# parse address from the webpage
def parse_address(date):
    regex_str = '[\u4E00-\u9FA5]+[0-9]*[\u4E00-\u9FA5]*[0-9]*[\u4E00-\u9FA5]*(?=[\uff0c|\u3002|\u3001|\uff08|\s])?'
    r = requests.get(result_sorted[result_sorted['date']==date]['link'].values[0], headers=headers)
    r.encoding = 'utf-8'
    soup = BeautifulSoup(r.text, 'lxml')
    address = []
    cleaned_address = pd.DataFrame()
    #for x in soup1.find('div', {'id':'ivs_content', 'class':'Article_content'}).find_all('p'):
    for x in soup.find_all('p'):
        if re.findall(regex_str, x.text):
            address += re.findall(regex_str, x.text)
    cleaned_address['address'] = address
    cleaned_address['date'] = date
    #cleaned_address['address'] = '上海市' + cleaned_address['address']
    return cleaned_address

# parse the address from refined address
def parse_json_address_(date):
    result = pd.DataFrame()
    result['country'] = [x['geocodes'][0]['country'] if len(x['geocodes'][0]['district']) != 0 else 'None' for x in real_address]
    result['province'] = [x['geocodes'][0]['province'] if len(x['geocodes'][0]['district']) != 0 else 'None' for x in real_address]
    result['citycode'] = [x['geocodes'][0]['citycode'] if len(x['geocodes'][0]['district']) != 0 else 'None' for x in real_address]
    result['city'] = [x['geocodes'][0]['city'] if len(x['geocodes'][0]['district']) != 0 else 'None' for x in real_address]
    result['district'] = [x['geocodes'][0]['district'] if len(x['geocodes'][0]['district']) != 0 else 'None' for x in real_address]
    result['location'] = [x['geocodes'][0]['location'] if len(x['geocodes'][0]['district']) != 0 else 'None' for x in real_address]
    result['long'] = result['location'].str.split(',', expand=True)[0]
    result['lat'] = result['location'].str.split(',', expand=True)[1]
    result['date'] = date
    assert cleaned_address['date'][0] == date, 'wrong date'
    result['address'] = cleaned_address[cleaned_address['date'] == date]['address']
    result.drop(result[result['country']=='None'].index, inplace=True)
    return result




In [13]:
tqdm.pandas()
date = '2022-04-26'
cleaned_address = parse_address(date)
cleaned_address

,address,date
0,微信号,2022-04-26
1,功能介绍,2022-04-26
2,生活在这座梦幻的城市,2022-04-26
3,与,2022-04-26
4,共同成长,2022-04-26
...,...,...
3971,各区卫健委,2022-04-26
3972,新闻办,2022-04-26
3973,编辑,2022-04-26
3974,林欣,2022-04-26


In [14]:
district_info = {'浦东新区':['板泉路2000弄', '紫薇路35弄'], '黄浦区':['白河路39号', '自忠路359号'], 
                  '静安区':['安远路515弄', '中兴路818号'], '徐汇区':['安福路198号', '中山西路1878弄'], 
                  '长宁区':['安化路311号', '遵义路820号'], '普陀区':['安远路188弄', '竹柳路50弄'], 
                  '虹口区':['安国路223弄', '株洲路399弄'], '杨浦区':['爱囯路301弄', '周家嘴路4500号'], 
                  '宝山区':['爱辉路28弄', '朱家弄村'], '闵行区':['爱博二村', '颛兴路666弄'], 
                  '嘉定区':['阿克苏路66弄', '震川路17号'], '金山区':['建农村', '建农村'], 
                  '松江区':['伴亭路855弄', '朱龙村'], '青浦区':['淀湖路145弄', '诸陆西路852号'],
                  '奉贤区':['海旗路1518号', '资福村'], '崇明区':['江南大道', '长凯路128号']}
district_names = list(district_info.keys())

In [15]:
# 分地区
index_start = []
index_end = []
for k in district_names:
  if district_info[k][0]:
    index_start.append(cleaned_address[cleaned_address['address'] == district_info[k][0]].index.values[0])
    index_end.append(cleaned_address[cleaned_address['address'] == district_info[k][1]].index.values[0])
  print(k)

浦东新区
黄浦区
静安区
徐汇区
长宁区
普陀区
虹口区
杨浦区
宝山区
闵行区
嘉定区
金山区
松江区
青浦区
奉贤区
崇明区


In [16]:
for i in range(len(index_start)):
  assert index_start[i] <= index_end[i]

In [17]:
cleaned_address_new = pd.DataFrame()
for i in range(len(index_start)):
    tmp = cleaned_address.loc[index_start[i]:index_end[i], :]
    tmp.loc[:, 'address'] = ['上海市' + district_names[i] + x for x in tmp.loc[:, 'address']]
    cleaned_address_new = pd.concat([cleaned_address_new, tmp], ignore_index=True)

C:\Users\johnn\Anaconda3\envs\pymc_env\lib\site-packages\pandas\core\indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value


In [18]:
print('The length of cleaned_address: {}'.format(len(cleaned_address_new)))

The length of cleaned_address: 3853


In [19]:
cleaned_address_new = cleaned_address_new[cleaned_address_new['address'] != '上海市长宁区住宅']

In [20]:
print('The length of cleaned_address: {}'.format(len(cleaned_address_new)))

The length of cleaned_address: 3833


In [21]:
cleaned_address_new.to_csv('address_4_26.csv', index=False)

,address,date
2821,上海市长宁区住宅,2022-04-15
2827,上海市长宁区住宅,2022-04-15
2907,上海市长宁区住宅,2022-04-15
2924,上海市长宁区住宅,2022-04-15
2973,上海市长宁区住宅,2022-04-15
2996,上海市长宁区住宅,2022-04-15
3064,上海市长宁区住宅,2022-04-15


In [21]:
# synchronize the data
def syn_data(df, database):
    df.set_index('address', inplace=True)
    database.set_index('address', inplace=True, verify_integrity=True)
    df.update(database)
    return df.reset_index()

In [73]:
# create the whole table
#all_names = ['address', 'date', 'country', 'province', 'citycode', 'city', 'district','location','long','lat']
#df_all = pd.DataFrame(columns=all_names)
#for i in range(13):
#  if i < 9:
#    tmp = pd.read_csv('address_4_0' + str(i+1) +'.csv')
#  else:
#    tmp = pd.read_csv('address_4_' + str(i+1) +'.csv')
#  df_all = df_all.append(tmp)


In [75]:
database = pd.read_csv('database_shanghai_address.csv')

In [76]:
df_all_new = syn_data(df_all, database)

In [20]:
missing_address_names = pd.DataFrame()
missing_address_names['address'] = missing_address['address']
missing_address_names['date'] = missing_address['date']

In [21]:
tqdm.pandas()
missing_address_names['json'] = missing_address_names['address'].progress_apply(lambda x: transformer(x))

  0%|          | 0/930 [00:00<?, ?it/s]

In [24]:
missing_address_all = parse_json_address_missing(missing_address_names)

In [26]:
len(database)

20973

In [35]:
database_new = pd.concat([database, missing_address_all.drop(columns=['date'])], ignore_index=True)

In [227]:
df_cur = df_cur[[x not in set(['上海市长宁区住宅', '上海市嘉定区号', '上海市嘉定区工地']) for x in df_cur['address']]]

In [229]:
df_cur.drop_duplicates(inplace=True)

C:\Users\johnn\Anaconda3\envs\pymc_env\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


# read data frame on the current date

In [22]:
# parse the address from refined address
def parse_json_address_missing(real_address_json):
    result = pd.DataFrame()
    result['country'] = [x['geocodes'][0]['country'] if len(x['geocodes'][0]['district']) != 0 else 'None' for x in real_address_json['json']]
    result['province'] = [x['geocodes'][0]['province'] if len(x['geocodes'][0]['district']) != 0 else 'None' for x in real_address_json['json']]
    result['citycode'] = [x['geocodes'][0]['citycode'] if len(x['geocodes'][0]['district']) != 0 else 'None' for x in real_address_json['json']]
    result['city'] = [x['geocodes'][0]['city'] if len(x['geocodes'][0]['district']) != 0 else 'None' for x in real_address_json['json']]
    result['district'] = [x['geocodes'][0]['district'] if len(x['geocodes'][0]['district']) != 0 else 'None' for x in real_address_json['json']]
    result['location'] = [x['geocodes'][0]['location'] if len(x['geocodes'][0]['district']) != 0 else 'None' for x in real_address_json['json']]
    result['long'] = result['location'].str.split(',', expand=True)[0]
    result['lat'] = result['location'].str.split(',', expand=True)[1]
    result['address'] = real_address_json['address']
    result['date'] = real_address_json['date']
    #assert cleaned_address['date'][0] == date, 'wrong date'
    #result['address'] = cleaned_address[cleaned_address['date'] == date]['address']
    result.drop(result[result['country']=='None'].index, inplace=True)
    #tables = pd.merge(tables_, result, on='address', how='left')
    return result

In [23]:
def retrieve_current_data(current_date):
    df_cur = pd.read_csv(current_date)
    df_cur_new = pd.merge(df_cur, database, on='address', how='left')
    missing_address = pd.DataFrame()
    missing_address['address'] = df_cur_new[df_cur_new.country.isna()]['address']
    missing_address['date'] = df_cur_new['date'][0]

    # Retrieve gps information for missing address
    tqdm.pandas()
    missing_address['json'] = missing_address.address.progress_apply(lambda x: transformer(x))

    # parse json data, update current date data
    missing_address.reset_index(inplace=True)
    missing_address_all = parse_json_address_missing(missing_address)
    missing_address_all.set_index(keys=['date', 'address'], inplace=True)
    df_cur_new.set_index(keys=['date', 'address'], inplace=True)
    df_cur_new.update(missing_address_all)
    df_cur_new.reset_index(inplace=True)

    return df_cur_new

# append the current date data
database = pd.read_csv('database_shanghai_address.csv')
df_all = pd.read_csv('covid_data_april.csv')
df_cur_new = retrieve_current_data('address_4_26.csv')
old_size = len(df_all)
#assert len(df_all) == 61045
df_all = pd.concat([df_all, df_cur_new], ignore_index=True)
df_all['date'] = pd.to_datetime(df_all['date'])
assert len(df_all) - len(df_cur_new) == old_size
assert all(df_all.country.notna())


  0%|          | 0/393 [00:00<?, ?it/s]

In [24]:
len(df_all.drop_duplicates())

125159

In [25]:
len(df_all)

125159

In [28]:
df_all.drop_duplicates(inplace=True)
print(len(df_all))

121326


In [26]:
df_all.to_csv('covid_data_april.csv', index=False)

In [27]:
## add to the unique address to database

database = pd.concat([database, df_cur_new.drop(columns='date')], ignore_index=True)
database.drop_duplicates(subset='address', inplace=True)
database

,address,country,province,citycode,city,district,location,long,lat,Unnamed: 0
0,上海市浦东新区张东路2281弄,中国,上海市,21.0,上海市,浦东新区,"121.637090,31.201005",121.63709,31.201005,NaN
1,上海市浦东新区临江村西新队罗汉松,中国,上海市,21.0,上海市,浦东新区,"121.481800,31.126070",121.4818,31.12607,NaN
2,上海市浦东新区临江村丁家队丁家宅,中国,上海市,21.0,上海市,浦东新区,"121.609684,31.181050",121.609684,31.18105,NaN
3,上海市浦东新区浦电路221号,中国,上海市,21.0,上海市,浦东新区,"121.523518,31.219127",121.523518,31.219127,NaN
4,上海市浦东新区龚路支路473弄,中国,上海市,21.0,上海市,浦东新区,"121.692128,31.263843",121.692128,31.263843,NaN
...,...,...,...,...,...,...,...,...,...,...
34377,上海市青浦区诸光路495号,中国,上海市,021,上海市,青浦区,"121.313417,31.165954",121.313417,31.165954,NaN
34379,上海市青浦区诸光路光联路,中国,上海市,021,上海市,青浦区,"121.302733,31.181764",121.302733,31.181764,NaN
34381,上海市奉贤区海旗路1518号,中国,上海市,021,上海市,奉贤区,"121.737708,30.895385",121.737708,30.895385,NaN
34383,上海市奉贤区金海公路3265号,中国,上海市,021,上海市,奉贤区,"121.499374,30.899830",121.499374,30.899830,NaN


In [31]:
database.to_csv('database_shanghai_address.csv', index=False)

# Covid map version 3.0

In [32]:
database = pd.read_csv('database_shanghai_address.csv')
df = pd.read_csv('covid_data_april.csv')

In [33]:
df['date'] = pd.to_datetime(df['date'])

In [34]:
# set time zone
tz_SH = pytz.timezone('Asia/Shanghai')
current_date = pd.to_datetime(datetime.now(tz_SH).date())
yesterday_date = current_date - pd.Timedelta(days=1)
days_7_date = current_date - pd.Timedelta(days=7)
print('Yesterday date:', yesterday_date.strftime('%Y/%m/%d'))
print('7 days ago date:', days_7_date.strftime('%Y/%m/%d'))

#
df_map = pd.DataFrame()
df_map['number_of_positive_days'] = df.groupby('address')['address'].count()
df_map['last_positive_date'] = df.groupby('address')['date'].max()
df_map['negative_days'] = [(current_date - x).days for x in df_map['last_positive_date']]
df_map.reset_index(inplace=True)

# check if there is positive case yesterday
yesterday_positive_list = set(df.loc[df['date'] == yesterday_date.strftime('%Y-%m-%d')]['address'])
days_7_positive_list = set(df.loc[(pd.to_datetime(df['date']) >= days_7_date) & (pd.to_datetime(df['date']) < current_date)]['address'])
df_map['yesterday_positive'] = [x in yesterday_positive_list for x in df_map['address']]
df_map['days_7_positive'] = [x in days_7_positive_list for x in df_map['address']]
assert sum(df_map['yesterday_positive']) == sum(df['date'] == yesterday_date.strftime('%Y-%m-%d'))

# merge location information
df_map = df_map.merge(database, on='address', how='left')

Yesterday date: 2022/04/25
7 days ago date: 2022/04/19


In [35]:
df.date.max().strftime(format="%Y-%m-%d")

'2022-04-25'

In [36]:
def fancy_html(row):
    i = row
    
    # obtain basic information
    address = df_map['address'].iloc[i]
    district = df_map['district'].iloc[i]                             
    last_postive_date = df_map['last_positive_date'].iloc[i]
    # obtain summaries
    yesterday_positive = "有" if df_map['yesterday_positive'].iloc[i] else '没有'
    days_7_positive = "有" if df_map['days_7_positive'].iloc[i] else '没有'
    number_of_positive = df_map['number_of_positive_days'].iloc[i]

    #if df_map['negative_days'].iloc[i] >= 7:
    #    district_type = '管控区'
    #elif df_map['negative_days'].iloc[i] >= 14:
    #    district_type = '防范区'
    #else:
    #    district_type = '封控区'
    
    left_col_colour = "#2A799C"
    right_col_colour = "#C5DCE7"
    
    html = """<!DOCTYPE html>
<html>

<head>
<h4 style="margin-bottom:0"; width="400px">当前日期:{}</h4>""".format(current_date.strftime('%Y/%m/%d')) + """

</head>
    <table style="height: 126px; width: 400px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">地址：</span></td>
<td style="width: 250px;background-color: """+ right_col_colour +""";">{}</td>""".format(address.split(district)[-1]) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">昨日是否有新增：</span></td>
<td style="width: 250px;background-color: """+ right_col_colour +""";">{}</td>""".format(yesterday_positive) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">7天内是否有新增：</span></td>
<td style="width: 250px;background-color: """+ right_col_colour +""";">{}</td>""".format(days_7_positive) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">最近新增阳性日期：</span></td>
<td style="width: 250px;background-color: """+ right_col_colour +""";">{}</td>""".format(last_postive_date.strftime('%Y/%m/%d')) + """
</tr>
</tbody>
</table>
<tfoot>
    <tr>
      <td colspan="2">Updated by {}""".format(df.date.max().strftime(format="%Y-%m-%d")) + """</td>
    </tr>
  </tfoot>
</html>
"""
    return html

In [37]:
mc = MarkerCluster(control=False)
# obtain number of days
total_days = (yesterday_date - pd.Timestamp('20220331')).days
for idx, row in df_map.iterrows():
    html = fancy_html(idx)
    iframe = branca.element.IFrame(html=html, width=400, height=200)
    popup = folium.Popup(iframe, parse_html=True)
    number_of_positive_days = df_map.loc[idx, 'number_of_positive_days']
    if row['negative_days'] >= 7:
        color = 'red'
    elif row['negative_days'] >= 14:
        color = 'lightred'
    else:
        color = 'darkred'
    mc.add_child(Marker([row['lat'], row['long']], popup=popup, 
                        icon=folium.Icon(color=color, icon='info-sign'),
                        name=row['address'].split(row['district'])[-1],
                        tooltip="点我查看详情！"))

In [38]:
covid_map = folium.Map(location=[31.230416, 121.473701], tiles=None, titles='shanghaimap', zoom_start=10)
folium.TileLayer(tiles='http://webrd0{s}.is.autonavi.com/appmaptile?lang=zh_cn&size=1&scale=1&style=8&x={x}&y={y}&z={z}', attr='&copy; 高德地图', name='上海疫情小区地图', subdomains='1234').add_to(covid_map)
covid_map.add_child(mc)
folium.plugins.LocateControl(strings={'title': '你的位置', 'popup': '你的位置'}, flyTo=True).add_to(covid_map)
Search(layer=mc, search_label='name', placeholder='搜索地址：', collapsed=False).add_to(covid_map)
embed_map(covid_map, "./map/index.html")

# Parse the positive and asymptomatic case 

In [66]:
link = 'http://wsjkw.sh.gov.cn/xwfb/20220416/dfbb15a0fe3b401dbead8b10995b3431.html'
r = requests.get(link, headers=headers)
r.encoding = 'utf-8'
soup = BeautifulSoup(r.text, 'lxml')

In [68]:
test = soup.find_all('strong')[0]

In [100]:
test = soup.find_all('strong')

In [115]:
 test[0].next_sibling.text.strip('，')

'居住于浦东新区'

In [121]:
for x in soup.find_all('strong'):
    print(x.next_sibling.text)

，居住于浦东新区，
，居住于黄浦区，
，居住于徐汇区，
，居住于长宁区，
，居住于静安区，
，居住于普陀区，
，居住于虹口区，
，居住于杨浦区，
，居住于闵行区，
，居住于宝山区，
，居住于嘉定区，
，居住于金山区，
，居住于松江区，
，居住于青浦区，
，居住于奉贤区，
，居住于崇明区，均为本市闭环隔离管控人员，其间新冠病毒核酸检测结果异常，经疾控中心复核结果为阳性。经市级专家会诊，综合流行病学史、临床症状、实验室检测和影像学检查结果等，诊断为确诊病例。
，居住于浦东新区，
，居住于黄浦区，
，居住于徐汇区，
，居住于长宁区，
，居住于静安区，
，居住于普陀区，
，居住于虹口区，
，居住于杨浦区，
，居住于闵行区，
，居住于宝山区，
，居住于嘉定区，
，居住于松江区，
，居住于青浦区，
，居住于奉贤区，
，居住于崇明区，
，居住于浦东新区，
，居住于黄浦区，
，居住于徐汇区，
，居住于长宁区，
，居住于静安区，
，居住于普陀区，
，居住于虹口区，
，居住于杨浦区，
，居住于闵行区，
，居住于宝山区，
，居住于嘉定区，
，居住于青浦区，
，居住于崇明区，
，居住于浦东新区，
，居住于黄浦区，
，居住于徐汇区，
，居住于长宁区，
，居住于静安区，
，居住于普陀区，
，居住于虹口区，
，居住于杨浦区，
，居住于闵行区，
，居住于宝山区，
，居住于嘉定区，
，居住于金山区，
，居住于松江区，
，居住于青浦区，
，居住于奉贤区，
，居住于崇明区，
，居住于浦东新区，
，居住于黄浦区，
，居住于徐汇区，
，居住于长宁区，
，居住于静安区，
，居住于普陀区，
，居住于虹口区，
，居住于杨浦区，
，居住于闵行区，
，居住于宝山区，
，居住于嘉定区，
，居住于松江区，
，居住于青浦区，
，居住于奉贤区，
，居住于崇明区，
均为中国籍，在美国探亲，自美国出发，乘坐同一航班，于2022年4月1日抵达上海浦东国际机场，入关后即被集中隔离观察，解除隔离前出现症状，即送指定医疗机构隔离排查。经专家组会诊，综合流行病学史、临床症状、实验室检测和影像学检查结果等，诊断为确诊病例。
为中国籍，在英国探亲，自英国出发，经丹麦转机，于2022年4月6日抵达上海浦东国际机场，入关后即被集中隔离观察，其间出现症状。综合流行病学史、临床症状、实验室检测和影像学检查结果等，诊断为确诊病例。
为中国籍，在巴西工作

AttributeError: 'NoneType' object has no attribute 'text'

In [95]:
regex = '(?:病例)(\d+)*(?:.)?(?:病例)?(\d*)' 
re.findall(regex, test)

[('2669', '2686')]

In [161]:
# parse case number 
def parse_case_number(regex, soup, date, type):
    df_positive = pd.DataFrame()
    positive_number = []
    positive_district = []

    for x in soup.find_all('strong'):
        if re.findall(regex, x.text):
            positive_number.append(re.findall(regex, x.text)[0])
            if x.next_sibling:
                tmp = re.findall('(?:居住于)([\u4E00-\u9FA5]+(?=\，))', x.next_sibling.text)
                if tmp:
                    positive_district.append(tmp[0])
        
    df_positive[type + '_case'] = positive_number[0:len(positive_district)]
    df_positive[type + '_number'] = [int(x[1]) - int(x[0]) + 1 if x[1] else int(1) for x in positive_number[0:len(positive_district)]]
    df_positive['district'] = positive_district 
    df_positive['date'] = date
    return df_positive


In [162]:
## positive number
regex = '(?:病例)(\d+)*(?:.)?(?:病例)?(\d*)' 
df_positive = parse_case_number(regex, soup, '2022/4/15', 'positive')

## asymptomatic number
regex = '(?:无症状感染者)(\d+)*(?:.)?(?:无症状感染者)?(\d*)'
df_asymptomatic = parse_case_number(regex, soup, '2022/4/15', 'asym')

In [163]:
df_positive

,positive_case,positive_number,positive_district,date
0,"(1, 924)",924,浦东新区,2022/4/15
1,"(925, 1156)",232,黄浦区,2022/4/15
2,"(1157, 1247)",91,徐汇区,2022/4/15
3,"(1248, 1302)",55,长宁区,2022/4/15
4,"(1303, 1384)",82,静安区,2022/4/15
5,"(1385, 1428)",44,普陀区,2022/4/15
6,"(1429, 1529)",101,虹口区,2022/4/15
7,"(1530, 1653)",124,杨浦区,2022/4/15
8,"(1654, 1724)",71,闵行区,2022/4/15
9,"(1725, 1742)",18,宝山区,2022/4/15


In [ ]:
def assign_group(df):
    split_points = df[df['positive_district'] == '浦东新区'].index.to_list()
    assert len(split_points) == 3
    group = ['A', 'B', 'C']
    for idx in split_points:
        df

In [180]:
df_positive[df_positive['positive_district'] =='浦东新区'].index.to_list()

[0, 16, 31]

In [182]:
df_positive.assign(types='A')

,positive_case,positive_number,positive_district,date,types
0,"(1, 924)",924,浦东新区,2022/4/15,A
1,"(925, 1156)",232,黄浦区,2022/4/15,A
2,"(1157, 1247)",91,徐汇区,2022/4/15,A
3,"(1248, 1302)",55,长宁区,2022/4/15,A
4,"(1303, 1384)",82,静安区,2022/4/15,A
5,"(1385, 1428)",44,普陀区,2022/4/15,A
6,"(1429, 1529)",101,虹口区,2022/4/15,A
7,"(1530, 1653)",124,杨浦区,2022/4/15,A
8,"(1654, 1724)",71,闵行区,2022/4/15,A
9,"(1725, 1742)",18,宝山区,2022/4/15,A
